In [1]:
import pandas as pd

In [39]:
df_data_sentence = pd.read_table("dictionary.txt")

In [40]:
df_data_sentence.head()

,!|0
0,! '|22935
1,! ''|18235
2,! Alas|179257
3,! Brilliant|22936
4,! Brilliant !|40532


In [41]:
df_data_sentence_processed = df_data_sentence["!|0"].str.split("|", expand=True)

In [42]:
df_data_sentence_processed.rename(columns={0: 'Phrase', 1: "phrase_ids"},inplace=True)

In [43]:
df_data_sentiment = pd.read_table("sentiment_labels.txt")

In [55]:
df_data_sentiment_processed = df_data_sentiment["phrase ids|sentiment values"].str.split("|", expand=True)

In [56]:
df_data_sentiment_processed = df_data_sentiment_processed.rename(columns={0: "phrase_ids", 1: "sentiment_values"})

In [59]:
df_processed_all = df_data_sentence_processed.merge(df_data_sentiment_processed, how='inner', on='phrase_ids')

In [60]:
list(df_processed_all.columns)

['Phrase', 'phrase_ids', 'sentiment_values']

In [61]:
df_processed_all.head()

,Phrase,phrase_ids,sentiment_values
0,! ',22935,0.52778
1,! '',18235,0.5
2,! Alas,179257,0.44444
3,! Brilliant,22936,0.86111
4,! Brilliant !,40532,0.93056


In [98]:
#WORD-EMBEDDINGS
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.layers import Bidirectional
from keras.layers import LSTM
from keras.layers import Dropout


In [63]:
df_processed_all['Phrase']

0                                                       ! '
1                                                      ! ''
2                                                    ! Alas
3                                               ! Brilliant
4                                             ! Brilliant !
                                ...                        
239226    zoning ordinances to protect your community fr...
239227                                            zzzzzzzzz
239228                                                 élan
239229                                                    É
239230                     É um passatempo descompromissado
Name: Phrase, Length: 239231, dtype: object

In [64]:
corpus = df_processed_all['Phrase'].to_list()

In [69]:
corpus[1024]

"'re down"

In [74]:
from keras.preprocessing.text import Tokenizer
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(corpus)

In [75]:
vocab_length = len(word_tokenizer.word_index) + 1

In [76]:
embedded_sentences = word_tokenizer.texts_to_sequences(corpus)
print(embedded_sentences)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [78]:
embedded_sentences[2024]

[6, 36, 1, 3854, 175, 4, 9]

In [79]:
#padding sentences so all batches have same dimensions
from nltk.tokenize import word_tokenize

word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(corpus, key=word_count)
length_long_sentence = len(word_tokenize(longest_sentence))

padded_sentences = pad_sequences(embedded_sentences, length_long_sentence, padding='post')


In [83]:
len(padded_sentences[22222])

56

In [84]:
#USING GLOVE
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open('glove.6B.100d.txt', encoding="utf8")

In [85]:
for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions

glove_file.close()

In [86]:
embedding_matrix = zeros((vocab_length, 100))
for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [133]:
embedding_matrix.shape

(18079, 100)

In [92]:
# model = Sequential()
# embedding_layer = Embedding(vocab_length, 100, weights=[embedding_matrix], input_length=length_long_sentence, trainable=False)
# model.add(embedding_layer)
# model.add(Flatten())
# model.add(Dense(1, activation='sigmoid'))

0.21110999584197998

In [209]:
model = Sequential()
embedding_layer = Embedding(vocab_length, 100, weights=[embedding_matrix], input_length=length_long_sentence, trainable=False)
model.add(embedding_layer)
model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)))

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.50))

model.add(Dense(1, activation='sigmoid'))

In [210]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (None, 56, 100)           1807900   
_________________________________________________________________
bidirectional_17 (Bidirectio (None, 256)               234496    
_________________________________________________________________
dense_29 (Dense)             (None, 512)               131584    
_________________________________________________________________
dropout_15 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 513       
Total params: 2,174,493
Trainable params: 366,593
Non-trainable params: 1,807,900
_________________________________________________________________
None


In [119]:
X_train = padded_sentences[:179231]

In [120]:
X_test = padded_sentences[179231:]

In [204]:
Y_train = sentiment_class[:179231]
Y_test = sentiment_class[179231:]

In [211]:
model.fit(X_train, Y_train, batch_size=1000, epochs=20)

Epoch 1/20
179231/179231 [==============================] - 21s 115us/step - loss: 0.5463 - acc: 0.7177
Epoch 2/20
179231/179231 [==============================] - 21s 115us/step - loss: 0.4986 - acc: 0.7549
Epoch 3/20
179231/179231 [==============================] - 21s 118us/step - loss: 0.4831 - acc: 0.7655
Epoch 4/20
179231/179231 [==============================] - 21s 119us/step - loss: 0.4701 - acc: 0.7745
Epoch 5/20
179231/179231 [==============================] - 22s 120us/step - loss: 0.4602 - acc: 0.7806
Epoch 6/20
179231/179231 [==============================] - 22s 122us/step - loss: 0.4519 - acc: 0.7868
Epoch 7/20
179231/179231 [==============================] - 22s 123us/step - loss: 0.4422 - acc: 0.7932
Epoch 8/20
179231/179231 [==============================] - 22s 124us/step - loss: 0.4354 - acc: 0.7976
Epoch 9/20
179231/179231 [==============================] - 22s 124us/step - loss: 0.4278 - acc: 0.8017
Epoch 10/20
179231/179231 [==============================] - 22s

In [161]:
df_processed_all['sentiment_values'].shape

(239231,)

In [182]:
sentiment = df_processed_all['sentiment_values'].to_list()

In [199]:
sentiment_class = []
for i in sentiment:
    i = float(i)
    if (i >= 0.5):
        sentiment_class.append(1)
    else:
        sentiment_class.append(0)

In [191]:
df_processed_all['sentiment_values'] = pd.to_numeric(df_processed_all['sentiment_values'])

In [203]:
sentiment_class[4]

1

In [194]:
df_processed_all.dtypes

Phrase               object
phrase_ids           object
sentiment_values    float64
dtype: object

In [212]:
scores = model.evaluate(X_test, Y_test, verbose=0)

In [213]:
scores

[0.4270751306504011, 0.8060166835784912]

In [214]:
scores[1]

0.8060166835784912